**Yapacağınız yorumlar için şimdiden teşekkür ederim. Çalışmayı beğenirseniz oylamayı unutmayın ^___^! Sorulara elimden geldiği kadarıyla cevap vermeye çalışacağım. Çalışmada sorulara ve geri bildirimlere göre düzeltme ve ekleme yapacağım.**

<a class="anchor" id="0."></a>
# **İçindekiler**

* [1. Çalışmanın Özeti](#1.)
* [2. Veri Seti Hakkında Kavramsal Değerlendirme](#2.)
* [3. Veri Seti Hakkında Teknik Değerlendirme](#3.)
* [4. Kusur Sütünlarının Bir Sütünda Toplanması](#4.)
* [5. Veri Setine Genel Bakış](#5.)
* * [5.1. Veri Setinin Boyutu](#5.1.)
* * [5.2. Veri Setindeki Bir Kaç Satır](#5.2.)
* * [5.3. Veri Setindeki Tüm Özelliklerin İstatiksel Özeti](#5.3.)
* * [5.4. Sınıf Dağılımları](#5.4.)
*  [6. Verilerin Grafiklerle İncelenmesi](#6.)
* * [6.1. Tek Değişken Dağılımı(Univariate Distribution) Grafikler](#6.1.)
* * [6.2. İki Değişken Dağılımı(Bivariate Distribution) Grafikler](#6.2.)
* [Yardımcı Fonksiyonlar](#55.)
* [SVC Sonuçları](#65.)
* [RandomForestClassifier Sonuçları](#75.)
* [Denetimsiz Öğrenmeyle Boyut Azaltma: Temel Bileşen Analizi](#76.)
* [İş Akışının Otomatikleştirilmesi](#77.)



[İçindekiler Menüsüne Git](#0.)
# <a class='anchor' id="1."></a>**1 Çalışmanı Özeti**

Bu çalışmada, çelik yüzeyindeki kusurların sınıflandırılması yapılmıştır.. Çalışmanın özeti şöyledir:

* Veri setinin yüklenmesi
* Sınıf etiketlerinin tek bir sütünda toplanması ve görselleştirme
* Destek Vektör Makineleri ve Rasgele Orman sınıflandırıcılarının en iyi parametre seçimi
* Sonuçların karışıklık matrisi ve sınıflandırma raporu ile verilmesi

**Yapacağınız yorumlar; çalışmanın daha iyi ve anlaşılır olmasını sağlayacaktır.**


In [ ]:
#Gerekli Kütüphanelerin Yüklenmesi Yapılıyor

import numpy as np # linear algebra
import pandas as pd # Veri işleme

#Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

#Makine Öğrenmesi 
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import train_test_split, GridSearchCV

#Sistem 
import os
import warnings

#Uyarıları kapatılıyor
warnings.filterwarnings('ignore')
print(os.listdir("../input"))

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="2."></a>**2. Veri Seti Hakkında Kavramsal Değerlendirme**

Bu veri seti, Semeion, İletişim Bilimleri Araştırma Merkezi tarafından yapılan araştırmalardan elde edilmiştir.. Araştırmanın asıl amacı, paslanmaz çelik plakalardaki yüzey kusurlarının tipini altı tip olası kusur (artı “diğer”) ile doğru bir şekilde sınıflandırmaktı. Özellik vektörü, kusurun geometrik şeklini ve taslağını yaklaşık olarak tanımlayan 27 özellikten oluşturulmuştur. Araştırma öyküsüne göre, Semeion bu görev için Centro Sviluppo Materiali (İtalya) kurumu tarafından görevlendirilmiştir ve bu nedenle özellik vektörleri olarak kullanılan 27 özelliğin niteliği veya 6 sınıf kusur tipi hakkında ayrıntılı bilgi vermek mümkün değildir.

Veri setinde 34 özellik vardır. İlk 27 özellik, görüntülerde görülen bir çeşit çelik levha hatasını içerir. 
Son 7 sütun, kodlanmış sınıflardır, yani plaka arızası "Stains" olarak sınıflandırılırsa, o sütunda 1 ve diğer sütunlarda 0 olur. Yani son 7 sınıf özellik vektörünün etiketidir.  Son yedi özellik şöyledir;
Pastry
Z_Scratch 
K_Scatch 
Stains
Dirtiness 
Bumps 
Other_Faults


In [ ]:
#Veri setinin yüklenmesi yapılıyor
dataset_temp=pd.read_csv("../input/faults.csv")
#Veri setine ait özet bilgiler 
dataset_temp.info()

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="3."></a>**3. Veri Seti Hakkında Teknik Değerlendirme**

Yukarıda yer alan bilglere bakarak veri seti hakkında şunları söylebiliriz:
* Veri seti 1941 örnekten oluşmaktadır. 
* Eksik veri yoktur
* Tüm veriler sayısaldır(noktalı sayı ve tamsayı ). 
* Son yedi sütun etiket sütunlarıdır

In [ ]:
#Veri setine ait ilk beş satır
dataset_temp.head()

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="4."></a>**4. Kusur Sütünlarının Bir Sütünda Toplanması**

'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps' ve 'Other_Faults' sütunları, örneğe ait etiketleri tutmaktadır. Bu etiket sütunlarını 'target' sütunu altında toplayalım

In [ ]:
#Kusur sınıfını gösteren sütünlar
label_columns=dataset_temp.columns.values[-7:]

#Kusur sınıfını gösteren sütünlar 'target' sütünu altında toplanıyor
targets=(dataset_temp.iloc[:,-7:]==1).idxmax(1)

#Kusur sınıfını gösterene sütünlar veri setinden çıkartılıyor
dataset=dataset_temp.drop(label_columns,axis=1)

#Oluşturulan target sınıfı veri setine ekleniyor
dataset['target']=targets
print("Kusur sütunları bir sütunda toplandı...")

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="5."></a>**5. Veri Setine Genel Bakış**

Veri setininin karakteristiği hakkında bilgiler edinmek makine öğrenmesi açısından faydalıdır. Bunun için veri seti hakkında aşağıdaki bilgilere göz atacağız:
* Veri Setinin Boyutu
* Veri Setindeki Bir Kaç Satır
* Veri Setindeki Tüm Özelliklerin İstatiksel Özeti
* Sınıf Değişkeninin Veriler Üzerindeki Dağılımı

Yukarıdaki bilgileri elde etmek için çoğu kez bir satır kod yazmak yeterli olacaktır.

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="5.1."></a>**5.1. Veri Setinin Boyutu**

* Veri setinde kaç örnek(satır) olduğu ve örneklerin özellik(sütun) sayılarını pandas DataFrame nesnesinin shape özlliğiyle öğrenebiliriz. 

In [ ]:
dataset.shape

Veri setinde 1941 örnek vardır ve her bir örneğin 27(çıktı değişkeni hariç) özelliğ vardır. 

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="5.2."></a>**5.2.Veri Setindeki Bir Kaç Satır**

Veri setindeki değerleri doğrudan incelemek her zaman fadalıdır. Bunun için pandas DataFrame nesnesinin **head()** fonksiyonundan faydalanacağız. **head()** fonksiyonu parametre olarak listelenecek satır sayısını alır ve birinci satırdan itibaren listeler. Varsayılan olarak listelenecek satır sayısı 5'tir;**head(n=5)**

In [ ]:
dataset.head(n=4)

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="5.3."></a>**5.3. Veri Setindeki Tüm Özelliklerin İstatiksel Özeti**

Veri setindeki özelliklerin istatiksel özeti Tanımlayıcı İstatistik(Descriptive Statistics) olarak tanımlanır. Tanımlayıcı istatistik veri setinin özetini ve analizini istatiksel hesaplamalarla ortaya koymayı amaçlar. Kullanılan temel hesaplamalarla; ortalama, standart sapma, mod, min, max, baslıklık ve eğrilik değerleri elde edilir. 

Tanımlayıcı istatiksel bilgileri elde etmek için pandas DataFrame nesnesinin **describe()** fonksiyonunu kullanacağız. **describe()** fonksiyonu sayısal(float int) özellikler için; count, mean, std, min, max  ve  yüzdelik değerlere nesne(string, timestamp) verileri için count, unique, top, ve freq değerlere döner. 

In [ ]:
describe=dataset.describe().T

"""
describe fonksiyonu yamukluk(skew) ve basıklık(kurtosis) değerlerini
vermediği için kendimiz ekliyoruz
"""
describe['skew']=dataset.skew().values
describe['kurtosis']=dataset.kurt().values
describe=round(describe, 2)
describe

Yukarıdaki tablo veri setinin özetini sunmakla birlikte, makine öğrenmesi açısında fazla bir şey ifade etmiyor.  Tablodan benim çıkarımım şöyledir:

* Özelliklerin değer aralıkları birbirinden çok farklı olduğu için eğer karar ağacı tabanlı yöntemler kullanılmayacaksa SVM, KNN, LR ve NN gibi, mutlaka ölçeklendirme yapılmalıdır. 

* skew ve kurtosis değerleri özelliklerin değer dağılımlarının normal dağılımdan uzak olduğunu gösteriyor.

Yukarıdaki tabloda veri setinde sayısal değer içeren tüm özelliklerin tanımlayıcı istatiksel bilgileri yer almaktadır. 'target' özelliği string değer içerdiği için tanımlayıcı istatiksel bilgileri yoktur. 'target' özelliğinin tanımlayıcı istatiksel bilgilerini ayrıca elde etmemiz gerekiyor. 

In [ ]:
dataset['target'].describe()

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="5.4."></a>**5.4. Sınıf Dağılımları**

Veri setindeki çıktı(sınıf, target) değişkeninin dağılımına bakalım. 

In [ ]:
print(dataset['target'].value_counts())

In [ ]:
#Sınıflara ait örnek sayılarının grafikle gösterimi 
fig, ax=plt.subplots(1,2,figsize=(15,6))
sns.countplot(x='target', data=dataset, ax=ax[0])
dataset['target'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax[1])

Bazı sınıfları diğer sınıflardan çok fazla örnek içeren veri setleri dengesiz dağılımlı(**Imbalanced**) veri setleri olarak adlandırılır. Dengesiz veri setleri üzerinde çalışırken daha kararlı sonuçlar elde etmek için dikkatli olmak gerekir. 

Yukarıdaki grafiklerde dengesiz bir sınıf dağılımı görünüyor. Veri setini eğitim ve test olarak ayrıştırdığımızda bu duruma dikkate etmemiz gerekiyor. Bunun için train_test_split fonksiyonunun stratify parametresini stratify=target olarak atamamız gerekir. Bu şekilde sınıflar **eşit oranda** eğitim ve test için ayrılacaktır. 

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="6."></a>**6. Verilerin Grafiklerle İncelenmesi** 

Bir önceki bölümde veri setinin tanımlayıcı istatiksel özet tablosunu elde etmiştik. İstatiksel özet tablosu veri seti hakkında çok fazla bilgi verse de bu bilgilerin makine öğrenmesi konularına aşina olmayan kişiler tarafından anlaşılması zordur. Verilerin  grafiklerle incelenmesi makine öğrenmesi uzmanı olmayan kişilerin verileri yorumlamasına imkan verdiği gibi makine öğrenmesinde uzman kişilerin veri daha iyi yorumlamasına da olanak sağlar. 

Verilerin incelenmesi için kullanılabilecek  grafikler türleri üçe ayrılır:
* Tek Değişken Dağılımı(Univariate Distribution) Grafikler
* İki Değişken Dağılımı (Bivariate Distribution) Grafikler
* Çoklu Değişken Dağılımı (Multivariate Distribution) Grafikler

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="6.1."></a>**6.1. Tek Değişken Dağılımı(Univariate Distribution) Grafikler** 

Tek değişken dağılımı için kullanılabilecek grafik türleri:
* Histogram
* Kernel Density Estimation
* Kutu Grafiği(BoxPlot)

**Histogram**

Histogram grafiği belirli aralıktaki değerlerin sıklığını kutularlar(bins) gösteren grafik türüdür. Bir özelliğin histogram grafiğine bakarak veri dağılımının normal(Gaussian), üstel, bimodal, yamuk(skewed) gibi karakteristiğini anlamaya olanak sağlar. 


In [ ]:
dataset.hist(figsize=(15,15))
plt.show()

Yukarıdaki histogram graklerine göre  dağılım karakteristiği belirgin özellikler:
* **Normal:** Empty_Index, Luminosity_Index
* **Yamuk:** Log_X_Index, Log_Y_Index
* **Üstel:** 'Edges_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Steel_Plate_Thickness', 'X_Maximum', 'X_Minimum', 'Y_Maximum', 'Y_Minimum',
* **Bimodal:** 'Length_of_Conveyer', 'Outside_Global_Index', 'TypeOfSteel_A300', 'TypeOfSteel_A400'

**Kernel Density Estimation(Çekirdek Yoğunluk Kestirimi)** 

KDE grafiği özelliğe ait verilen dağılımı gösteren tek değişken dağılım grafiğidir. Histogram grafiğindeki kutuların üst kısımlarının birbirine çizgiyle birleştirilmesiyle olşturulur. İnsan gözünün yorumlamasına daha elverişlidir. Histogram grafiği gibi, belki daha iyi, verinin normal(Gaussian), üstel, bimodal(iki tepeli), yamuk(skewed) dağılım karakteristiğini anlamaya olanak sağlar. 

In [ ]:
dataset.plot(kind="density", layout=(6,5), 
             subplots=True,sharex=False, sharey=False, figsize=(15,15))
plt.show()

Yukarıdaki yoğunluk grafiklerinde dağılım türü histogram grafiklerinden daha açık görülebilmektedir. Özellikle bimodal dağılımlar daha açık görülüyor. 

**Box Plot(Kutu Çizim) Grafiği**

Kutu  grafiği tek değişken dağılımı için kullanılabilecek diğer bir grafik türüdür. Kutu grafiği, her bir özelliğin istatiksel dağılımını özetler, medyan (orta değer) için bir çizgi çizer ve 25 ve 75. persentillerin arasındaki değerleri kutuyle temsil eder(verilerin% 50'si). Kutunun alt ve üstünden çizilen T çizimleriyle(Bıyık olarak adlandırılır), verilerin yayılması hakkında fikir verir ve bıyıkların dışındaki noktalar aykırı değerleri gösterir (verilerin% 50'sinin yayılma boyutundan 1.5 kat daha büyük değerler aykırı değer olarak kabul edilir).

In [ ]:
dataset.plot(kind='box', layout=(6,5), 
             subplots=True,sharex=False, sharey=False, figsize=(15,15) )
plt.show()

Yukarıdaki kutu grafiklerinde Y_Minimum, Y_Maximimum, Maximum_Luminosity, vb. özelliklerinde aykırı değerler açıkca görülüyor. 

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="6.2."></a>**6.2. İki Değişken Dağılımı(Bivariate Distribution) Grafikler** 

Hedef değişkenle özelliklerin arasındaki dağılıma bakarak makine öğrenmesi modeline yaptıkları katkıları yorumlayabiliriz.

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren farklı grafikler;
#boxplot, violinplot, pointplot, barplot
def draw_bivariate_plot(dataset, rows, cols, plot_type):
    column_names=dataset.columns.values
    number_of_column=len(column_names)
    fig, axarr=plt.subplots(rows,cols, figsize=(22,16))

    counter=0
    for i in range(rows):
        for j in range(cols):
            if column_names[counter]=='target':
                break
            if 'violin' in plot_type:
                sns.violinplot(x='target', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='target', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='target',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='target',y=column_names[counter],data=dataset, ax=axarr[i][j])
                
            counter+=1
            if counter==(number_of_column-1,):
                break

In [ ]:
draw_bivariate_plot(dataset=dataset, rows=6, cols=5,plot_type="box")

Yukarıdaki grafik yer darlığından dolayı sıkışık görünüyor. En iyi özellikleri seçip tekrardan görselleştirelim. 

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
X=dataset.drop('target',axis=1)
Y=le.fit_transform(dataset['target'])

In [ ]:
kbest=SelectKBest(score_func=f_classif, k=6)
kbest.fit(X,Y)

In [ ]:
k_best=kbest.get_support(indices=True)
feature_names=dataset.columns.values
k_best_feature_names=feature_names[k_best]
k_best_feature_names =np.append(k_best_feature_names,"target")

In [ ]:
draw_bivariate_plot(dataset=dataset[k_best_feature_names], 
                    rows=2, cols=3,plot_type="box")

[İçindekiler Menüsüne Git](#0.)
## <a class="anchor" id="55."></a>**Yardımcı Fonksiyonlar**

**draw_confusion_matrix** fonksiyonu karışıklık matrisini çizdirmek için kullanılacaktır

**MyGridSearch** fonksiyonu sınıflandırıcıların en iyi sonuç veren parametrelerini bulmak için kullanılacaktır

In [ ]:
#Karışıklık matrisisinin çizimi
def draw_confusion_matrix(cm):
    plt.figure(figsize=(12,8))
    sns.heatmap(cm,annot=True,fmt="d") 
    plt.title("Confusion Matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
def get_train_test(dataset):
    #Kullanılan veri seti makine öğrenmesi modeli için uygun hale getiriliyor
    #x: bağımsız değişkenler
    #y: bağımlı değişken
    X=dataset.drop('target', axis=1)
    y=dataset['target']
    
    #Veri setindeki özellikler farklı değer aralıklarına sahip olduğu için
    #ölçeklendirmenin yapılması gerekir
    standardScaler=StandardScaler()
    X=standardScaler.fit_transform(X)
    
    #Veri seti eğitim ve test bölümlerine ayrıştırılıyor
    #Eğitim veri seti kullanılarak model eğitilecek
    #Test veri setinde eğitilen veri seti test edilecektir
    X_train,X_test,y_train,y_test = train_test_split(X,#veri matrisi
                                                     y, #çıktı sütün vektörü
                                                     stratify=y,#Sınıf oranları dikkate alnıyor
                                                     test_size = 0.3,#%70 eğitim, %30 test
                                                     random_state = 42)
    return X_train,X_test,y_train,y_test

In [ ]:
def MyGridSearch(clf, param_grid, dataset):
    
    print("Sınıflandırıcı:",clf.__class__.__name__)
    
    #Eğitim ve test bölümleri alınıyor
    X_train,X_test,y_train,y_test=get_train_test(dataset)
    
    #GridSearchCV ile çapraz doğrulama yapılır
    #Eğitim veri seti beş(cv=5) parçaya ayrılarak her birisi için başarı oranları elde edilir.
    grid_search=GridSearchCV(clf, param_grid, cv=5)
    
    #Beş ayrı bölüme ayrılan eğitim setinin her birisi için eğitim gerçekleştirilir
    grid_search.fit(X_train, y_train)

    #Bel ayrı başarı oranından en iyisine göre test veri setinin sonucu elde edilir.
    accuracy=grid_search.score(X_test,y_test)
    
    #Test başarı oranı yazdırılıy
    print("Başarı oranı:",accuracy)
    #Test için kullanılan parametreler yazdırılıyor
    print("grid_search.best_params_:",grid_search.best_params_)
    
    #Karışıklık matrisi ve sınıflandırma raporu için tüm test verisine ait
    #tahminler yapılıyor
    y_pred=grid_search.predict(X_test)
    
    cm = confusion_matrix(y_test,y_pred)
    draw_confusion_matrix(cm)
    print('Sınıflandırma Raporu: \n',classification_report(y_test,y_pred))


[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="65."></a> **SVC Sonuçları** 

In [ ]:
svc_param_grid={'C':[0.001,0.01,0.1, 1, 10, 100],
           'gamma':[0.001,0.01,0.1, 1, 10, 100],
               'kernel':['linear', 'rbf']}
#Bu işlem bir kaç dakika kadar sürebiliyor. İsterseniz çalışmayı çatallayıp(Fork) deneyebilirsiniz.
#Elde edilen en iyi parametreler; {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
#MyGridSearch(SVC(), svc_param_grid, dataset )

In [ ]:

svc_best_params={'C':[ 10],
           'gamma':[0.1],
            'kernel':['rbf']}
MyGridSearch(SVC(), svc_best_params, dataset )

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="75."></a>**RandomForestClassifier Sonuçları** 

In [ ]:
random_forest_param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
#Bu işlem beş dakika kadar sürebiliyor.
#Elde edilen en iyi parametreler; 
# {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
#MyGridSearch(RandomForestClassifier(n_estimators=48), random_forest_param_grid, dataset )

In [ ]:
rfc_best_params={'n_estimators':[48],
    'bootstrap': [False], 
             'criterion': ['entropy'], 
             'max_depth': [None], 
             'max_features': [10], 
             'min_samples_leaf': [1], 
             'min_samples_split': [2]}
MyGridSearch(RandomForestClassifier(), rfc_best_params, dataset )

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="76."></a>**Denetimsiz Öğrenmeyle Boyut Azaltma: Temel Bileşen Analizi** 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
 X_train,X_test,y_train,y_test=get_train_test(dataset)

In [ ]:
pca=PCA(whiten=True)
pca.fit(X_train)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(pca.explained_variance_)

Yukarıdaki grafif 15 temel bileşenin özellik verktörünü temsil edebileceğini göstermektedir. 15 temel bileşene göre PCA dönüşümünü yapalım

In [ ]:
pca=PCA(n_components=15)
X_train_pca=pca.fit_transform(X_train)
X_test_pca=pca.transform(X_test)
print("PCA dönüşümü gerçekleştirildi")

In [ ]:
clf=SVC()
clf.fit(X_train_pca, y_train)
y_pred=clf.predict(X_test_pca)
print("PCA dönüşümü yapılan veri seti üzerinde tahmin yapıldı")

In [ ]:
cm = confusion_matrix(y_test,y_pred)
draw_confusion_matrix(cm)
print("Başarı oranı:{:.2f}".format(clf.score(X_test_pca,y_test)))
print('Sınıflandırma Raporu: \n',classification_report(y_test,y_pred))

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="77."></a>**İş Akışının Otomatikleştirilmesi** 


sklearn, makine öğrenmesi uygulamalarının standart olan iş akışını otomatikleştimek için Pipeline(boru hattı) nesnesini sunar. Pipeline kullanılarak ön işleme, özellik çıkarma, boyut azaltma ve sınıflandırma işlemleri tek bir nesneyle kontrol edilebilir. 

Pipeline eğitim ve  test verilerinde aynı işlemlerin yapılmasına garanti ederek makine öğrenmesi performansının tutarlı olmasına yardımcı olur.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print("Model eğitildi ve tahmin yapıldı...")

In [ ]:
cm = confusion_matrix(y_test,y_pred)
draw_confusion_matrix(cm)
print("Başarı oranı:{:.2f}".format(pipeline.score(X_test,y_test)))
print('Sınıflandırma Raporu: \n',classification_report(y_test,y_pred))